In [1]:
"""
generate train data, validation data, and test data
"""
import os
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.ndimage import imread
from scipy.misc import imsave
from scipy.misc import imresize
import numpy as np
import os
from numpy import genfromtxt
import tensorflow as tf
from PIL import Image
import pickle
import helper
from tensorflow_vgg import vgg16
from tensorflow_vgg import utils
import collections
import time

# Get dataset


In [24]:
from numpy import genfromtxt
IDList_A_55_probe = genfromtxt('OULP-C1V2_Pack/OULP-C1V2_SubjectIDList(FormatVersion1.0)/IDList_OULP-C1V2-A-75_gallery.csv',
           delimiter=',', dtype=np.int32)

print('length:',len(IDList_A_55_probe))
for row in IDList_A_55_probe:
    print(row)

length: 3759
[ 24   0 102 133]
[ 32   0  79 108]
[ 35   0  97 128]
[36  0 63 95]
[37  0 49 80]
[ 38   0  92 122]
[ 39   0  81 115]
[ 40   0  92 122]
[ 41   0  88 116]
[ 42   0  89 119]
[ 43   0  86 118]
[ 44   0  93 124]
[ 45   0  80 109]
[ 46   0  92 122]
[ 47   0 120 151]
[48  0 67 98]
[ 49   0 101 133]
[ 50   0  99 129]
[ 51   0 103 133]
[ 52   0  92 121]
[ 53   0  90 119]
[ 54   0 118 151]
[ 55   0 131 164]
[ 56   0  71 103]
[ 57   0 114 145]
[ 58   0  87 117]
[ 59   0  98 129]
[ 60   0 114 145]
[ 61   0 106 136]
[ 62   0 106 134]
[ 63   0 133 165]
[ 64   0 111 141]
[ 65   0  89 120]
[ 66   0  91 126]
[ 67   0 124 157]
[ 68   0  99 129]
[ 69   0  97 126]
[ 70   0  92 120]
[ 71   0 109 141]
[ 72   0 107 137]
[ 73   0 119 150]
[ 74   0 121 154]
[ 75   0  88 120]
[1000001       0     149     178]
[1000002       0     156     187]
[1000003       0      97     121]
[1000004       0      69      99]
[1000006       0     108     139]
[1000007       0      91     119]
[1000008       0     

In [269]:
def generate_gait_cycle_list(folder_path, csv_file):
    for subject in csv_file:
        images_list = np.ndarray((subject[3] - subject[2] + 1, 224, 224, 3))
    
        subject_dir = str(subject[0])
        if len(subject_dir) != 7:
            subject_dir = '0' * (7 - len(subject_dir)) + subject_dir
                
        for ii, image_name in enumerate(range(subject[2], subject[3]+1)):        
            image_name = str(image_name)
            if len(image_name) == 2:
                image_name = '0' + image_name
            elif len(image_name) == 1:
                image_name = '00' + image_name
            images_list[ii, :, :, :] = imread(os.path.join(folder_path, subject_dir, '00000{}.png'.format(image_name)))
            
            if ii == 0:
                begin_image = image_name
                
        yield (images_list, subject_dir, begin_image, image_name)

In [107]:
Or_folder_path = 'OULP-C1V2_Pack/OULP-C1V2_NormalizedSilhouette(224x224x3)/Seq00'
# subject_list = os.listdir(Or_folder_path) 
# subject_list = subject_list[1:]
# save_folder = 'OULP-C1V2_Pack/OULP-deference-(128x88)/Seq00'
image_list_tuple = generate_gait_cycle_list(Or_folder_path, IDList_A_55_probe)

total = []
for ii in range(len(IDList_A_55_probe)//10):
    next_input = next(image_list_tuple)
    total.append(next_input[0])

# Extract CNN features

In [109]:
total[0].shape

(32, 224, 224, 3)

In [14]:
codes = None

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    vgg = vgg16.Vgg16()
    input_ = tf.placeholder(tf.float32, [None, 224, 224, 3])
    with tf.name_scope("content_vgg"):
        vgg.build(input_)
    
        for ii, subject in enumerate(data_list_gallery, 1):
            feed_dict = {input_: subject}
            codes_batch = sess.run(vgg.relu6, feed_dict=feed_dict)
            codes_batch = np.array(codes_batch)
            
            if codes is None:
                codes = codes_batch
            else:
                codes = np.concatenate((codes, codes_batch))

            print('{} subjects processed'.format(ii))

/Users/congcong/Desktop/Gait/OULP-Database/tensorflow_vgg/vgg16.npy
npy file loaded
build model started
build model finished: 0s
1 subjects processed
2 subjects processed
3 subjects processed
4 subjects processed
5 subjects processed
6 subjects processed
7 subjects processed
8 subjects processed
9 subjects processed
10 subjects processed
11 subjects processed
12 subjects processed
13 subjects processed
14 subjects processed
15 subjects processed
16 subjects processed
17 subjects processed
18 subjects processed
19 subjects processed
20 subjects processed
21 subjects processed
22 subjects processed
23 subjects processed
24 subjects processed
25 subjects processed
26 subjects processed
27 subjects processed
28 subjects processed
29 subjects processed
30 subjects processed
31 subjects processed
32 subjects processed
33 subjects processed
34 subjects processed
35 subjects processed
36 subjects processed
37 subjects processed
38 subjects processed
39 subjects processed
40 subjects processed


In [15]:
codes.shape

(11396, 4096)

In [28]:
codes_new = []
codes_new.append(codes[:total[0].shape[0]])
length_begin = 0
length_end = 32
for idx in range(len(total) - 1):
    length_begin += total[idx].shape[0]
    length_end += total[idx+1].shape[0]
    codes_new.append(codes[length_begin:length_end])

# 归一化

In [166]:
maxi = np.max(np.amax(codes, axis=1))
mini = np.min(np.amin(codes, axis=1))

In [167]:
codes_new = (codes_new - mini)/(maxi - mini)

In [168]:
codes_new = np.array(codes_new)
with open('gait_data/codes', 'wb') as f:
    np.save(f, codes_new)

## Extract Test Features

# Generate 44x44 target

In [270]:
total[0].shape[1]

224

In [271]:
targets_44x44 = []
x = []

for each in total:
    for frame in each:
        x.append(imresize(frame[:,:,0], [44,44], interp='nearest'))
    targets_44x44.append(np.array(x))
    x = []

In [272]:
targets_44x44 = np.array(targets_44x44)/255

In [273]:
with open('gait_data/targets_44x44_375batch', 'wb') as f:
    np.save(f, targets_44x44)

# Flatten target

In [274]:
targets_flatten = []
x = []

for each in total:
    for frame in each:
        x.append(imresize(frame[:,:,0], [44,44], interp='nearest').flatten())
    targets_flatten.append(np.array(x))
    x = []

In [275]:
targets_flatten_44x44 = np.array(targets_flatten)/255

In [276]:
with open('gait_data/targets_flatten_44x44', 'wb') as f:
    np.save(f, targets_flatten_44x44)

# Checkpoint 1 Read features targets

In [2]:
data_list_gallery = np.load(open(r'gait_data/codes_normalized', mode='rb'))

In [3]:
targets = np.load(open(r'gait_data/targets_flatten_44x44_375batch', mode='rb'))

# get_subject_batches

In [4]:
def get_all_degree(train_or_val_x):
    """
    把所有subject的degree拆开组堆成一列
    """
    total_subject = []
    for subject in train_or_val_x:
        total_subject.append(np.array(subject))
    total_subject = np.array(total_subject)
    
    return total_subject

def get_batches(train, target, batch_size):
# def get_30_batches(ii, subject):
    """
    yield every batch inside of one subject
    [1, 2, 3, ..., 29] <---> [2, 3, 4, ..., 30]
    :params ii: idx
    subject: a list of subject with its all images in one degree.
    """
    data_length = len(train)
    n_batches = data_length//batch_size
    train = train[:n_batches * batch_size]
    target = target[:n_batches * batch_size]
    
    for ii in range(n_batches):
        train_batch = []
        target_batch = []
        train_batch = train[ii: ii + batch_size]
        target_batch = target[ii: ii + batch_size]
#         for jj in new_batch:
#             for kk in jj:
                
#             train_batch = np.ndarray([1, 28,11264], dtype=np.float32)
#             for idx, each in enumerate(subject[:28]):
#                 train_batch[0, idx, :] = np.array(each).flatten()
        
        yield (train_batch, target_batch)

def process_batch(train_batch, target_batch):
    """
    形成length_sequence, 形成train_batch
    """
    seq_length = []
    train = np.zeros([train_batch.shape[0], 41, 4096], dtype=np.float32)
    target = np.zeros([target_batch.shape[0], 41, 1936], dtype=np.float32)
    for ii, each in enumerate(train_batch):
        ii_length = each.shape[0] - 1
        seq_length.append(ii_length)
        train[ii, :ii_length, :] = each[:-1]
    for ii, each in enumerate(target_batch):
        ii_length = each.shape[0] - 1
        target[ii, :ii_length, :] = each[1:]
        
    return (train, target, seq_length)
             
        #     if ii == 0:
    #         out_batch = np.ndarray([1, 15, 11264], dtype=np.float32)
    #         for idx, each in enumerate(subject[:15]):
    #             each = np.array(each).flatten()
    #             out_batch[0, idx, :] = each
    #     else:
    #         out_batch = np.ndarray([1, 1, 11264], dtype=np.float32)
    #         out_batch[0, 0, :] = np.array(subject[ii + 14]).flatten()
    #     yield out_batch
    

# process_batch(next(get_batches(x, 20))

# Get train, validation, test dataset

In [5]:
print("data length: {}".format(len(data_list_gallery)))
print("every length has : {}".format(np.array(data_list_gallery[0]).shape))

data length: 375
every length has : (32, 4096)


In [6]:
split_frac = 0.8
split_num = int(split_frac*len(data_list_gallery))

train_x_raw, val_x_raw = data_list_gallery[:split_num], data_list_gallery[split_num:]
train_y_raw, val_y_raw = targets[:split_num], targets[split_num:]
# val_test_split = len(val_x_raw)//2
# val_x_raw, test_x_raw = val_x_raw[:val_test_split], val_x_raw[val_test_split:]
# val_y_raw, test_y_raw = val_y_raw[:int(len(val_y_raw)/2)], val_y_raw[int(len(val_y_raw)/2):]

In [7]:
print("Train_x: {}".format(np.array(train_x_raw).shape))
print("Validation_x: {}".format(np.array(val_x_raw).shape))
print("Train_y: {}".format(np.array(train_y_raw).shape))
print("Validation_y: {}".format(np.array(val_y_raw).shape))

Train_x: (300,)
Validation_x: (75,)
Train_y: (300,)
Validation_y: (75,)


# Build the graph

In [24]:
def build_rnn(batch_size=20, lstm_size=1000, num_layers=1, learning_rate=0.001, grad_clip=1):
    
    tf.reset_default_graph()
#     graph = tf.Graph()
#     with graph.as_default():
    with tf.name_scope('inputs'):
        inputs_ = tf.placeholder(tf.float32, [None, None, 4096], name="inputs")
    with tf.name_scope('targets'):
        target_ = tf.placeholder(tf.float32, [None, None, 1936], name="targets")

    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    seq_length = tf.placeholder(tf.int32, (None, ), name='seq_length')
    
    with tf.name_scope("RNN_cells"):
        # Your basic LSTM cell
#         lstm = tf.contrib.rnn.LSTMBlockFusedCell(lstm_size, forget_bias=1.0, use_peephole=True)
#         lstm = tf.nn.relu(lstm)
        lstm = tf.contrib.rnn.GRUCell(lstm_size, activation=tf.nn.relu)
        # Add dropout to the cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        # Stack up multiple LSTM layers, for deep learning
        cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
        
    with tf.name_scope("RNN_init_state"):
        # Getting an initial state of all zeros
        initial_state = cell.zero_state(batch_size, tf.float32)
    
    with tf.name_scope("RNN_forward"):
        outputs, final_state = tf.nn.dynamic_rnn(cell, inputs_, initial_state=initial_state, sequence_length=seq_length)

    with tf.name_scope('predictions'):
        predictions = tf.contrib.layers.fully_connected(outputs, 1936, activation_fn=tf.sigmoid)
        tf.summary.histogram('predictions', predictions)
        
    with tf.name_scope('cost'):
        cost = tf.losses.mean_squared_error(target_, predictions)
        tf.summary.scalar('cost', cost)
        
    with tf.name_scope('train'):
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip, name='clip')
        train_op = tf.train.AdamOptimizer(learning_rate)
        optimizer = train_op.apply_gradients(zip(grads, tvars))
        
    with tf.name_scope('saver'):
        saver = tf.train.Saver()
        
    merged = tf.summary.merge_all()
    
    export_nodes = ['inputs_', 'target_', 'initial_state', 'final_state',
                    'keep_prob', 'cost', 'predictions', 'optimizer', 'merged', 
                    'saver', 'learning_rate', 'seq_length', 'cell']
    
    Graph = collections.namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

# Accuracy

In [25]:
def calculate_accuracy(sess, predictions, target_, seq_length):
    error = abs(target_ - predictions)
    mean = []
    for ii, each_row in enumerate(error):
        each_row = each_row[:seq_length[ii]]
        row_mean = np.sum(each_row <= 0.15) / each_row.size
        mean.append(row_mean)
        
    accuracy = np.sum(np.array(mean)) / len(mean) * 100
    
    return accuracy

# Train

In [26]:
# train_x = get_all_degree(train_x_raw)
# val_x = get_all_degree(val_x_raw)
train_x = train_x_raw
val_x = val_x_raw
train_y = train_y_raw
val_y = val_y_raw

In [27]:
len(val_y)

75

In [28]:
def train(model, epochs, file_writer, save_string):

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        iteration = 1
        mean_val_Acc = 0
        mean_val_loss = 0
        count_Acc_not_increase_epochs = 0
        count_loss_not_decrease_epochs = 0
        Last_val_Acc = 0
        Last_val_loss = 0
        
        for e in range(epochs):
            for train_batch_1, target_batch_1 in get_batches(train_x, train_y, batch_size):
                start = time.time()
                
                state = sess.run(model.initial_state)

                train_batch, target_batch, sq_length = process_batch(train_batch_1, target_batch_1)

    #             print('target: {}'.format(target_batch.shape))
                feed = {model.inputs_: train_batch,
                        model.target_: target_batch,
                        model.keep_prob: 0.5,
                        model.initial_state: state,
                        model.learning_rate: lr,
                        model.seq_length: sq_length
                        }

                loss, _, pred, summary = sess.run([model.cost, model.optimizer, model.predictions, model.merged], \
                                              feed_dict=feed)
#                 print('pred: {}'.format(type(pred)))
    #             print('pred: {}'.format(np.count_nonzero(pred)))
    #             print('pred: {}'.format(np.sum(pred>0.1)))
    #             print('pred: {}'.format(pred.size))
    #             print('output: {}'.format(out[1, 39, :]))
                if iteration%5==0:
                    end = time.time()
                    acc = calculate_accuracy(sess, pred, target_batch, sq_length)
                    print("Epoch: {}/{},".format(e, epochs),' ',
                          "Iteration: {},".format(iteration),' ',
                          "Train loss: {:.3f},".format(loss),' ',
                          "{:.1f}s /batch.".format((end-start)/5),' ',
                          "ACCuracy: %{:.3f}".format(acc))
                    
                    file_writer.add_summary(summary, iteration)
                
                if iteration%25==0:
                    validation_loss = []
                    validation_Acc = []
                    
                    if batch_size >= len(val_x):
                        val_batch_size = 30
                    else: 
                        val_batch_size = batch_size
                    
                    for val_batch_1, val_target_batch_1 in get_batches(val_x, val_y, val_batch_size):

                        val_state = sess.run(model.cell.zero_state(val_batch_size, tf.float32))

                        val_batch, val_target_batch, val_sq_length  = process_batch(val_batch_1, val_target_batch_1)

                        feed = {model.inputs_: val_batch,
                                model.target_: val_target_batch,
                                model.keep_prob: 1,
                                model.initial_state: val_state,
                                model.seq_length: val_sq_length
                               }

                        val_loss, val_pred = sess.run([model.cost, model.predictions], feed_dict=feed)
                        
                        val_acc = calculate_accuracy(sess, val_pred, val_target_batch, val_sq_length)

                        validation_loss.append(val_loss)
                        
                        validation_Acc.append(val_acc)
                        
                    Last_val_Acc = mean_val_Acc
                    Last_val_loss = mean_val_loss
                        
                    mean_val_loss = sum(validation_loss)/len(validation_loss)  
                    mean_val_Acc = sum(validation_Acc)/len(validation_Acc)
                    print()
                    print("Validation loss: {:.3f},".format(mean_val_loss), ' ',
                          "Validation Accuracy: %{:.3f}".format(mean_val_Acc))
                    print()
                    
                iteration += 1
                
            # Early stopping  
            if mean_val_Acc - Last_val_Acc <= -0.5:
                count_Acc_not_increase_epochs += 1
            if Last_val_loss - mean_val_loss <= -0.01:
                count_loss_not_decrease_epochs += 1
            
            if count_Acc_not_increase_epochs >= 10:
                break
            if count_loss_not_decrease_epochs >= 20:
                break
                
        model.saver.save(sess, r"{}".format(save_string))

In [29]:
!mkdir checkpoints

mkdir: checkpoints: File exists


In [31]:
epochs = 1000
batch_size = 50
for lstm_size in [1000]:
    for num_layers in [2]:
        for lr in [0.0013]:
                log_string = 'logs/4/lr={},nl={},ls={},bs={}'.format(lr, num_layers, lstm_size, batch_size)
                save_string = 'checkpoints\lr={}_nl={}_ls={}_bs={}.ckpt'.format(lr, num_layers, lstm_size, batch_size)
                
                writer = tf.summary.FileWriter(log_string)

                model = build_rnn(batch_size=batch_size, lstm_size=lstm_size, 
                                  num_layers=num_layers, learning_rate=lr, grad_clip=1)

                train(model, epochs, writer, save_string)
                
                print(' ')
                print(' ')
                print("leraning_rate={},num_layers={},lstm_size={},batch_size={} finished, saved".format(lr, num_layers, lstm_size, batch_size))
                print(' ')
                print(' ')

Epoch: 0/1000,   Iteration: 5,   Train loss: 0.240,   2.1s /batch.   ACCuracy: %1.311
Epoch: 1/1000,   Iteration: 10,   Train loss: 0.148,   2.0s /batch.   ACCuracy: %60.590
Epoch: 2/1000,   Iteration: 15,   Train loss: 0.136,   2.0s /batch.   ACCuracy: %74.151
Epoch: 3/1000,   Iteration: 20,   Train loss: 0.133,   1.9s /batch.   ACCuracy: %79.015
Epoch: 4/1000,   Iteration: 25,   Train loss: 0.131,   2.0s /batch.   ACCuracy: %80.288

Validation loss: 0.140,   Validation Accuracy: %80.218

Epoch: 4/1000,   Iteration: 30,   Train loss: 0.131,   2.0s /batch.   ACCuracy: %81.982
Epoch: 5/1000,   Iteration: 35,   Train loss: 0.130,   2.0s /batch.   ACCuracy: %82.241
Epoch: 6/1000,   Iteration: 40,   Train loss: 0.129,   2.1s /batch.   ACCuracy: %82.275
Epoch: 7/1000,   Iteration: 45,   Train loss: 0.127,   2.1s /batch.   ACCuracy: %83.210
Epoch: 8/1000,   Iteration: 50,   Train loss: 0.127,   2.1s /batch.   ACCuracy: %84.350

Validation loss: 0.135,   Validation Accuracy: %83.789

Epoch: 9

KeyboardInterrupt: 

In [163]:
for train_batch_1, target_batch_1 in get_batches(train_x, train_y, batch_size):
#     train_batch_1, target_batch_1 = 
    print(train_batch_1[7].shape)
    train_batch, target_batch, sq_length = process_batch(train_batch_1, target_batch_1)
    print(sq_length)

(31, 4096)
[31, 29, 31, 32, 31, 30, 34, 30, 28, 30, 32, 31, 29, 30, 31, 31, 32, 30, 30, 29, 29, 33, 33, 32, 31, 30, 31, 31, 30, 28]
(29, 4096)
[29, 31, 32, 31, 30, 34, 30, 28, 30, 32, 31, 29, 30, 31, 31, 32, 30, 30, 29, 29, 33, 33, 32, 31, 30, 31, 31, 30, 28, 32]
(31, 4096)
[31, 32, 31, 30, 34, 30, 28, 30, 32, 31, 29, 30, 31, 31, 32, 30, 30, 29, 29, 33, 33, 32, 31, 30, 31, 31, 30, 28, 32, 30]
(33, 4096)
[32, 31, 30, 34, 30, 28, 30, 32, 31, 29, 30, 31, 31, 32, 30, 30, 29, 29, 33, 33, 32, 31, 30, 31, 31, 30, 28, 32, 30, 31]
(32, 4096)
[31, 30, 34, 30, 28, 30, 32, 31, 29, 30, 31, 31, 32, 30, 30, 29, 29, 33, 33, 32, 31, 30, 31, 31, 30, 28, 32, 30, 31, 35]
(30, 4096)
[30, 34, 30, 28, 30, 32, 31, 29, 30, 31, 31, 32, 30, 30, 29, 29, 33, 33, 32, 31, 30, 31, 31, 30, 28, 32, 30, 31, 35, 33]
(31, 4096)
[34, 30, 28, 30, 32, 31, 29, 30, 31, 31, 32, 30, 30, 29, 29, 33, 33, 32, 31, 30, 31, 31, 30, 28, 32, 30, 31, 35, 33, 30]
(32, 4096)
[30, 28, 30, 32, 31, 29, 30, 31, 31, 32, 30, 30, 29, 29, 33, 33, 

In [173]:
0.1*255

25.5